<h1>Data Processing</h1>

Ziel dieses Notebooks ist, die gescrapeten Daten so zu verarbeiten, dass Sie später thematisch durchsucht werden können. Ein Embeddingvektor liefert genau diese Funktion. Auf eine Anfrage hin wird der Abstand zwischen dem Embedding der Frage und den Embeddings aller anderen Dokumenten berechnet. Dann werden die Dokumente mit den kleinsten Abständen ausgewählt und dem LLM als Kontext mitgegeben. Mithilfe des Wissens dieser Dokumente soll dass LLM dann in der Lage sein die Frage korrekt zu beantworten.
Für die Embeddings benutzen wir [Google Bert](https://blog.google/products/search/search-language-understanding-bert/)

Beispiel:

User: Welche Dozenten unterrichten das Fach Grundlagen der Informatik?

System wählt besten 5 Dokumente aus 

    <Dokument 1>: ... betreute Prof. Dr. Löhr eine Batchelorarbeit in Grundlagen der Informatik...
    <Dokument 2>: Prof. Dr. Weber tel.: 013882664 email: weber@th.de Raum: HQ: 403, Fächer: Grundlagen der Informatik ...
    <Dokument 3> ...
    <Dokument 4> ...
    <Dokument 5> ...
    

Aus der Nutzeranfrage und den Dokumenten wird eine neue Query erstellt, die dem LLM dann final bereitgetellt wird. Diese sieht in etwa so aus:

    
    <Dokument 1>: ... betreute Prof. Dr. Löhr eine Batchelorarbeit in Grundlagen der     Informatik...
    <Dokument 2>: Prof. Dr. Weber tel.: 013882664 email: weber@th.de Raum: HQ: 403, Fächer: Grundlagen der Informatik ...
    <Dokument 3> ...
    <Dokument 4> ...
    <Dokument 5> ...

    Bitte beantworte folgende Frage unter der Berücksichtigung obiger Dokumente:
    Welche Dozenten unterrichten das Fach Grundlagen der Informatik?


Das LLM wird daraufhin hoffentlich korrekt eine Antwort liefern die ähnlich ist zu:

    A: An der TH Nürnberg Georg Simon Ohm unterichten die Professoren Prof. Dr. Löhr und Prof. Dr. Weber das Fach Grundlagen der Informatik.


In [1]:
import sqlite3
import pandas as pd
from transformers import BertModel, BertTokenizer
import torch
from tqdm import tqdm
from scipy.spatial.distance import cosine
from db_init import db_get_df, db_save_df
import json

/Users/br/Projects/IT-Ptojekt-Chatbot/daibl/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Zunächst laden wir die Daten aus der Datenbank. Dabei besitzt jedes Dokument als Metadaten den Titel der Webseite, den filenamen und den Text. Diese speichern wir uns in einen Pandas Dataframe

In [3]:
df = db_get_df("html_attrs", ["filename", "title", "text"])

print(df.dtypes)
print(df["text"][3])

filename    object
title       object
text        object
dtype: object
        Nuremberg Tech is a university with strong regional roots that understands its role in a globalized living, employment, and research community. Our aim is to offer our students degree programmes and a learning environment that provide access to applied research in today’s international context. Therefore, an international and intercultural orientation already forms a key component of life at the university; the development of this characteristic is anchored in our internationalisation strategy .  The following third-party funded projects support the implementation and continued development of the measures described in the strategy to advance internationality at Nuremberg Tech.                           Internationalisation squared (2022 – 2023)  The Internationalisation squared (INT 2 ) project implements and further develops Nuremberg Tech’s internationalisation strategy. Based on a comprehensive analysis o

Zur überprüfung der Texte können wir nun einmal eine Keywordsuche starten. Dieser Ansatz wird außerdem tiefer im Notebook [spacy_keywordextraction](./spacy_keywordextraction.ipynb) verfolgt.

In [4]:
word = "Gallwitz"

[text for text in df["text"] if word in text][:5]

['       Voraussetzungen / Zielgruppe      Welche Studiengänge gibt es an der Fakultät Informatik?         Welche Studiengänge gibt es an der Fakultät Informatik?  Wirtschaftsinformatik Medieninformatik Informatik abgeschlossenes Bachelorstudium in - Wirtschaftsinformatik - Information Systems and Management - Information Technology in Business Computing - Computer Science in Business Computing abgeschlossenes Bachelorstudium in - Medieninformatik - Informatik - Computer Science - Information Technology abgeschlossenes Bachelorstudium in - Informatik - Computer Science - Information Technology mindestens 210 ECTS-Punkte, Nachqualifikation möglich mindestens 210 ECTS-Punkte, Nachqualifikation möglich mindestens 210 ECTS-Punkte, Nachqualifikation möglich Sechssemestrige Bachelorstudiengänge entsprechen 180 ECTS-Punkten. Daher sind im Masterstudium zusätzliche 30 ECTS-Punkte nachzuholen. Fehlende ECTS-Punkte können durch Nachqualifikation ausgeglichen werden. Zugangsvoraussetzung für den 

Jetzt werden wir für jedes Dokument ein eigenes Word embeddings erstellen. Dazu müssen wir zunächst das BERT Model laden.
Das BERT Model ist ein von Google trainiertes mehrschichtiges neuronales Netz, welches ursprünglich dafür entwickelt wurde, dass ???
es ist trainiert auf 10.000+ Büchern
es gibt Modelle "base" und "large"
uncased heißt ohne klein - Großschreibung

Wir brauchen zur vorbereitung die zusätzlichen Token
[SEP] um das Ende eines Satzes zu markieren
[CLS] am Anfang des Texten
[PAD] zum auffüllen der Token 
Außerdem
TokenIDs
MaskIDs - zum filtern der [PAD]
Segment IDs um verschiedene Sätze zu unterscheiden
Posititional Embeddings


In [5]:
#TODO try better model
model = BertModel.from_pretrained('bert-base-uncased',output_hidden_states = True) 
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
testSentence = "In der Bibliothek gibt es 40 Bücher zum Thema Animes"
tokens_question = tokenizer.tokenize(testSentence)
print(*tokens_question)

in der bi ##bl ##iot ##he ##k gi ##bt es 40 bu ##cher zu ##m them ##a anime ##s


Nun erstellen wir eine neue Spalte ["tokens"], in der wir für jedes Dokument die Tokens abspeichern.

In [7]:
df["tokens"] = [tokenizer.tokenize(text) for text in tqdm(df["text"])]

  0%|          | 0/2433 [00:00<?, ?it/s]

100%|██████████| 2433/2433 [00:50<00:00, 47.83it/s] 


Der dataframe hat nun eine Spalte mehr und wir können uns ein Beispiel der Tokens ansehen.

In [9]:
print(df.dtypes)
print(df["tokens"][2])

filename    object
title       object
text        object
tokens      object
dtype: object
['eine', 'pilots', '##tu', '##die', '.', 'oder', ':', 'we', '##r', 'ku', '##mmer', '##t', 'sic', '##h', 'um', 'das', 'eh', '##ren', '##am', '##t', '?', 'in', 'der', 'pr', '##ax', '##is', 'engagement', '##ford', '##ern', '##der', 'st', '##ru', '##kt', '##ure', '##n', 'lei', '##sten', '„', 'vera', '##nt', '##wo', '##rt', '##liche', 'fur', 'engagement', '“', 'in', 'ko', '##mm', '##une', '##n', ',', 've', '##re', '##inen', 'und', 'verb', '##and', '##en', 'eine', '##n', 'wi', '##cht', '##igen', 'beit', '##rag', 'zu', '##m', 'gel', '##ingen', 'des', 'engagements', '.', 'se', '##it', 'lange', '##rem', 'ist', 'dies', 'u', '.', 'a', '.', 'ein', 'fe', '##ld', 'der', 'professional', '##isi', '##er', '##ung', 'fur', 'abs', '##ol', '##vent', ':', 'inn', '##en', 'des', 'stud', '##ien', '##gang', '##s', 'so', '##zia', '##le', 'ar', '##bei', '##t', '.', 'in', 'der', 'bis', '##her', '##igen', 'for', '##sch', '##un

Diese Tokens müssen nun in IDs umgewandelt werden, damit sie das BERT Model für die erstellung eines Embedding Vectors benutzen kann. Dafür benutzen wir eine Funktion des Tokenizers convert_tokens_to_id.

In [10]:
df["token_ids"] = [tokenizer.convert_tokens_to_ids(tokens) for tokens in tqdm(df["tokens"])]

100%|██████████| 2433/2433 [00:03<00:00, 749.38it/s]


In [12]:
special_symbols = ["[CLS]", "[SEP]"]
print(tokenizer.convert_tokens_to_ids(special_symbols))


[101, 102]


Wir haben jetzt also die Tokens IDs für unsere 2400 verschiedenen Dokumente gebildet. Der nächste Schritt wäre nun, diese Tokens IDs in dem BERT Model zu übergeben, sodass es uns ein Embedding daraus errechnet. Leider kann das BERT Model nur 512 Tokens (~1300 Zeichen) als Input nehmen. Die meißten der gescrapeten Webseiten sind aber wesentlich länger. Der Naheliegendste Ansatz ist dabei, die Tokens einfach in 510 token große Chunks aufzusplitten (Wir brauchen noch 2 Tokens extra für jeden Chunk) und für jeden Chunk ein extra Embedding zu erstellen. Dabei gibt es entweder die Möglichkeit die Chunks überlappend, oder einfach hard cut zu gestalten. Wir werden hier zunächst den hard cut Ansatz verfolgen. 

In [16]:
# splice dokuments in 512 token chunks

# Initialize an empty list to store rows for the new DataFrame
new_rows = []

# Function to split text and tokens into chunks of 512 tokens
def split_text_and_tokens(row):
    text = row['text']
    tokens_ids = row['token_ids']
    filename = row['filename']

    if len(tokens_ids) > 510:
        # Split into multiple chunks
        for i in range(0, len(tokens_ids), 510):
            chunk_tokens = tokens_ids[i:i + 510]
            # adding the [CLS] and the [SEP] token
            chunk_tokens = [101] + chunk_tokens + [102]
            chunk_text = tokenizer.decode(chunk_tokens)

            # Create a new row with a reference to the original row
            new_row = {'filename': filename, 'chunk_id': i/510, 'chunk_text': chunk_text, 'chunk_tokens_json': json.dumps(chunk_tokens)}
            new_rows.append(new_row)
    else:
        # If the row has 510 tokens or fewer, keep it as is

        # adding the [CLS] and the [SEP] token
        chunk_tokens = [101] + tokens_ids + [102]
        text = "[CLS]" + text + "[SEP]"
        new_row = {'filename': filename, 'chunk_id': 0, 'chunk_text': text, 'chunk_tokens_json': json.dumps(tokens_ids) }
        new_rows.append(new_row)

# Apply the function to each row in the original DataFrame
df.apply(split_text_and_tokens, axis=1)

# Create a new DataFrame from the list of new rows
chunk_df = pd.DataFrame(new_rows)

# Reset the index of the new DataFrame if needed
chunk_df.reset_index(drop=True, inplace=True)

# Print the new DataFrame
print(chunk_df.sample(2).to_markdown())

# tokenizer.convert_tokens_to_string


|      | filename                                                                    |   chunk_id | chunk_text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

Ab jetzt werden wir mit dem neuen chunk_df weiterarbeiten. Nun erstellen wir für jeden Chunk ein eigenes Embedding, welches dann die Semantik dieses chunks enthalten soll. Dafür müssen wir uns nun das BERT Model etwas genauer anschauen.

In [ ]:
def proccessSentence(tokens, model, tokenizer):
    if len(tokens) == 0:
        # Handle the case when the token list is empty, for example, return a default embedding or raise an exception.
        # For demonstration purposes, we'll return a zero tensor as the default embedding.
        return torch.zeros(768)

    tokens = ["CLS"] + tokens + ["SEP"]

    attention_mask = [1 if token != "[PAD]" else 0  for token in tokens]
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    token_ids_tensor = torch.tensor([token_ids], dtype=torch.int64)
    attetion_mask_tensor = torch.tensor([attention_mask], dtype=torch.int64)

    with torch.no_grad():
        outputs = model(token_ids_tensor, attetion_mask_tensor)
        hidden_states = outputs[2]

    # stack the layer list 
    token_embeddings = torch.stack(hidden_states, dim=0)
    # remove the batches dim
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    # Swap dimensions 0 and 1.
    token_embeddings = token_embeddings.permute(1,0,2)
    # average all token embeds
    layer_vecs = torch.mean(token_embeddings, dim=0)



    # Calculate the average of layer 3 to 13
    embed = torch.mean(layer_vecs[2:], dim=0)


    return embed


df["chunk_embeddings_json"] = [proccessSentence(tokens).tolist() for tokens in tqdm(df["tokens"])]


Diser Code kann sehr lange brauchen um die Embeddings zu berechnen. An dieser Stelle sollte man dann die Embeddings am besten abspeichern. 

In [ ]:
db_save_df(df, "chunk_embeddings")

In [10]:
df = db_get_df("chunk_embeddings", ["filename", "chunk_word_embeddings_2"])

In [11]:
import ast
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE 
import numpy as np  

df = df['chunk_word_embeddings_2'].copy()



# Verwenden Sie 'ast.literal_eval', um Zeichenketten in Listen umzuwandeln
df = df.apply(ast.literal_eval)

# Wandeln Sie die Listen von Listen in ein Numpy-Array um
word_embeddings = np.array(df.tolist())

tsne = TSNE(n_components=2, perplexity=30, n_iter=300)
X_embedded = tsne.fit_transform(word_embeddings)
plt.figure(figsize=(10, 6))
plt.scatter(X_embedded[:, 0], X_embedded[:, 1], s=5)
plt.title("t-SNE Visualization of Word Embeddings")
plt.xlabel("t-SNE Dimension 1")
plt.ylabel("t-SNE Dimension 2")
plt.show() 


: 

In [ ]:
#dokument--> bert anwenden für jeden dokument
# question-bert anwenden
question="was macht Gallwitz?"
document=df["text"][1]

tokens_question = tokenizer.tokenize(question)
tokens_document = tokenizer.tokenize(document)
attetion_mask_question = [1] * len(tokens_question)
attention_mask_dokument = [1] * len(tokens_document)

token_idss = tokenizer.convert_tokens_to_ids(tokens_question)
tokenDocument_idss = tokenizer.convert_tokens_to_ids(tokens_document)


tokens_tensor = torch.tensor([token_idss])
segments_tensors = torch.tensor([attetion_mask_question])

tokensDocument_tensor = torch.tensor([tokenDocument_idss])
segmentsDocument_tensors = torch.tensor([attention_mask_dokument])

with torch.no_grad():
    outputs = model(tokens_tensor, segments_tensors)
    hidden_states = outputs[2]

with torch.no_grad():
    outputs = model(tokensDocument_tensor, segmentsDocument_tensors)
    hiddenDocuments_states = outputs[2]

# print(token_idss)
# print(tokenDocument_idss)

print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0

print ("Number of batches:", len(hidden_states[layer_i]))
batch_i = 0

print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))


Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 1
Number of tokens: 6
Number of hidden units: 768


In [ ]:
token_embeddings = torch.stack(hidden_states, dim=0)
token_embeddings = torch.squeeze(token_embeddings, dim=1)
token_embeddings = token_embeddings.permute(1,0,2)

token_vecs_sum = []

# For each token in the sentence...
for token in token_embeddings:
    
    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))

In [ ]:
token_vecs = hidden_states[-2][0]

# Calculate the average of all 22 token vectors.
sentence_embedding = torch.mean(token_vecs, dim=0)
print ("Our final sentence embedding vector of shape:", sentence_embedding)


tokenDocuments_vecs = hiddenDocuments_states[-2][0]

# Calculate the average of all 22 token vectors.
sentenceDocument_embedding = torch.mean(tokenDocuments_vecs, dim=0)
print ("Our final sentence embedding vector of shape:", sentence_embedding)


In [ ]:


# Calculate the cosine similarity between the word bank 
# in "bank robber" vs "river bank" (different meanings).
diff_bank = 1 - cosine(sentence_embedding, sentenceDocument_embedding)

print('Vector similarity for *different* meanings:  %.2f' % diff_bank)

In [ ]:
import json

print(len(df["text"][4]))
# print(json.loads(df["text"][0]))
diff_bank = 1 - cosine(json.loads(df["word_embeddings"][3]), json.loads(df["word_embeddings"][6]))
# for embed in df["word_embeddings"]:
# print(embed)

print(diff_bank)

In [ ]:
df["text"][1746]

In [ ]:
word = "Fachhochschulgesetz"

df.loc[df["text"].str.contains(word)]["text"]
# [text for text in df["text"] if word in text][:5]

In [ ]:
from question_embedding import question_embeddings
import json
import matplotlib.pyplot as plt


# TODO 10 Fragen
# TODO TSNE

df = db_get_df("chunk_word_embeddings_all")
print(df.dtypes)
question = "TH"
question_embedding = question_embeddings(question)

df["distance"] = [1 - cosine(json.loads(embedding), question_embedding) for embedding in df["chunk_word_embeddings"]]
most_similar_documents = df.nsmallest(5, "distance")
print(f"question embedding: {question_embedding[:10]}")
print(most_similar_documents["distance"])


df["distance"].plot(kind='hist', bins=200)
plt.show()

In [ ]:
df = db_get_df("word_embeddings", ["filename", "title", "text", "tokens"])
df["token_ids"] = [tokenizer.convert_tokens_to_ids(json.loads(tokens)) for tokens in df["tokens"]]

In [ ]:
# splice dokuments in 512 token chunks

# Initialize an empty list to store rows for the new DataFrame
new_rows = []

# Function to split text and tokens into chunks of 512 tokens
def split_text_and_tokens(row):
    text = row['text']
    tokens_ids = row['token_ids']
    filename = row['filename']

    if len(tokens_ids) > 512:
        # Split into multiple chunks
        for i in range(0, len(tokens_ids), 512):
            chunk_tokens = tokens_ids[i:i + 512]
            chunk_text = tokenizer.decode(chunk_tokens)

            # Create a new row with a reference to the original row
            new_row = {'filename': filename, 'chunk_id': i/512, 'chunk_text': chunk_text, 'chunk_tokens_json': json.dumps(chunk_tokens)}
            new_rows.append(new_row)
    else:
        # If the row has 512 tokens or fewer, keep it as is
        new_row = {'filename': filename, 'chunk_id': 0, 'chunk_text': text, 'chunk_tokens_json': json.dumps(tokens_ids) }
        new_rows.append(new_row)

# Apply the function to each row in the original DataFrame
df.apply(split_text_and_tokens, axis=1)

# Create a new DataFrame from the list of new rows
new_df = pd.DataFrame(new_rows)

# Reset the index of the new DataFrame if needed
new_df.reset_index(drop=True, inplace=True)

# Print the new DataFrame
print(new_df.to_markdown())

# tokenizer.convert_tokens_to_string


Aus den 2433 Dokumenten die wir eigentlich gescraped haben, sind nun 6945 chunks entstanden es hat sich fast verdreifacht. Wenn man die 787 Seiten ohne Inhalt abzieht, hat sich die Anzahl von 1646 auf 6158 fast vervierfacht.